In [22]:
import shutil
from pathlib import Path

gastrovision_mini_demo_path = Path("demos/gastrovision_mini")

if gastrovision_mini_demo_path.exists():
    shutil.rmtree(gastrovision_mini_demo_path)

gastrovision_mini_demo_path.mkdir(parents=True, exist_ok=True)

gastrovision_mini_examples_path = gastrovision_mini_demo_path / "examples"
gastrovision_mini_examples_path.mkdir(parents=True, exist_ok=True)

gastrovision_mini_examples = [
    Path("/home/glauco/Desktop/projects/learningPyTorch/data/pizza_steak_sushi_20_percent/test/pizza/2782998.jpg"),
    Path("/home/glauco/Desktop/projects/learningPyTorch/data/pizza_steak_sushi_20_percent/test/steak/289822.jpg"),
    Path("/home/glauco/Desktop/projects/learningPyTorch/data/pizza_steak_sushi_20_percent/test/sushi/804460.jpg")
]

for example in gastrovision_mini_examples:
    destination = gastrovision_mini_examples_path / example.name
    shutil.copy2(src=example, dst=destination)

effnetb2_gastrovision_min_model_path = "/home/glauco/Desktop/projects/learningPyTorch/trained_models/09_pretrained_effnetb2_20_percent.safetensors"
effnetb2_gastrovision_min_model_destionation = gastrovision_mini_demo_path / effnetb2_gastrovision_min_model_path.split("/")[-1]
shutil.copy2(src=effnetb2_gastrovision_min_model_path, dst=effnetb2_gastrovision_min_model_destionation)

PosixPath('demos/gastrovision_mini/09_pretrained_effnetb2_20_percent.safetensors')

In [23]:
%%writefile demos/gastrovision_mini/model.py
import torchvision

from torch import nn

def create_effnet2_model(num_classes: int=3):
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b2(weights=weights)

    for param in model.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes)
    )

    return model, transforms

Writing demos/gastrovision_mini/model.py


In [24]:
%%writefile demos/gastrovision_mini/app.py
import os
import torch
import gradio as gr

from typing import Tuple, Dict
from model import create_effnet2_model
from safetensors.torch import load_file
from timeit import default_timer as timer

class_names = ["pizza", "steak", "sushi"]

effnetb2, effnetb2_transforms = create_effnet2_model(num_classes=3)
sd = load_file("09_pretrained_effnetb2_20_percent.safetensors")
missing, unexpected = effnetb2.load_state_dict(sd, strict=True)



def predict(img) -> Tuple[Dict, float]:
    start_time = timer()

    img = effnetb2_transforms(img).unsqueeze(dim=0)

    effnetb2.eval()
    with torch.inference_mode():
        pred_probs = torch.softmax(effnetb2(img), dim=1)

    # Gradio's required format: a dictionary with the probability for each class
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    pred_time = round(timer()-start_time, 5)

    return pred_labels_and_probs, pred_time



title = "🍕🥩🍣 Vision Model"
description = "An EfficientNetB2 feature extractor computer vision model able to differentiate photos of pizza, steak and sushi"
article = "Created by Glauco Masi"

example_list = [["examples/"+example] for example in os.listdir("examples")]

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Label(num_top_classes=3, label="Predictions"),
        gr.Number(label="Prediction time (s)")
    ],
    examples=example_list,
    title=title,
    description=description,
    article=article
)

demo.launch()

Writing demos/gastrovision_mini/app.py


In [25]:
import torch
import gradio
import torchvision
import safetensors
print(torch.__version__)
print(torchvision.__version__)
print(gradio.__version__)
print(safetensors.__version__)

2.8.0+cu128
0.23.0+cu128
5.44.1
0.6.2


In [28]:
%%writefile demos/gastrovision_mini/requirements.txt
torch==2.8.0
torchvision==0.23.0
gradio==5.44.1
safetensors==0.6.2

Overwriting demos/gastrovision_mini/requirements.txt
